In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping


In [2]:
def create_sequences(df, feature_cols, target_col, window):
    X, y = [], []

    arr = df[feature_cols].values      # convert ONCE
    target_arr = df[target_col].values # convert ONCE

    for i in range(len(df) - window):
        X.append(arr[i:i+window])
        y.append(target_arr[i+window])

    return np.array(X), np.array(y)


In [ ]:
import pandas as pd
df = pd.read_csv("E://PHD/Course materials/Sem 1/Data Mining/Assignment/workspace/stock_price_trend_prediction/stock_price_pred_data_mining/Data/05_cluster_output/cluster_output_Kmeans.csv")
# 1. Compute next-week (5 trading days) forward return per stock
df['next_week_close'] = df.groupby('stock_id')['Close'].shift(-5)
df['next_week_return'] = (df['next_week_close'] - df['Close']) / df['Close']
# 2. Binary target: 1 = uptrend, 0 = downtrend or flat
df["target"] = (df["Close"].shift(-5) > df["Close"]).astype(int)
# 3. Drop rows where we can't compute future return (last 5 days per stock)
df = df.dropna(subset=['next_week_return'])
# Optional: drop helper column
df = df.drop(columns=['next_week_close'])
drop_cols = ['Date', 'stock_id', 'agg_cluster', 'target', 'next_week_return']
feature_cols = [c for c in df.columns if c not in drop_cols]

print("Number of features:", len(feature_cols))
print("Features:", feature_cols)

Number of features: 23
Features: ['Open', 'High', 'Low', 'Close', 'Volume', 'MA7', 'MA21', 'EMA20', 'EMA50', 'RSI_14', 'MACD', 'MACD_signal', 'MACD_hist', 'BB_upper', 'BB_middle', 'BB_lower', 'CCI_14', 'CMF_20', 'Stoch_K', 'Stoch_D', 'Momentum_10', 'Daily_Return', 'Log_Return']


In [5]:
df.shape

(72548, 28)

In [ ]:
from sklearn.metrics import roc_auc_score

WINDOW = 20  # recommended
cluster_lstm_models = {}

for cl in df['agg_cluster'].unique():

    print(f"\n🔵 Training LSTM for Cluster {cl}")
    cluster_df = df[df['agg_cluster'] == cl].copy().sort_values(["stock_id", "Date"])
    
    X_all, y_all = [], []

    # ---- SEQUENCE BUILDING PER STOCK (memory-safe)
    for sid, temp in cluster_df.groupby("stock_id"):
        if len(temp) <= WINDOW + 2:
            continue
        X_temp, y_temp = create_sequences(temp, feature_cols, 'target', WINDOW)
        X_all.append(X_temp)
        y_all.append(y_temp)

    if not X_all:
        print(f"No usable data for cluster {cl}")
        continue

    # Combine results
    X = np.vstack(X_all)
    y = np.concatenate(y_all)

    print(f"Cluster {cl}: Final sequence shape = {X.shape}")

    # ---- Train-test split (time-based)
    split_idx = int(0.7 * len(X))
    X_train, X_test = X[:split_idx], X[split_idx:]
    y_train, y_test = y[:split_idx], y[split_idx:]

    # ---- Build LSTM Model
    model = Sequential([
        LSTM(64, return_sequences=False, input_shape=(WINDOW, len(feature_cols))),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # ---- Fit model
    es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    model.fit(
        X_train, y_train,
        validation_split=0.2,
        epochs=30,
        batch_size=128,
        callbacks=[es],
        verbose=1
    )

    # ---- Store model
    cluster_lstm_models[cl] = model

    # ---- Evaluate
    y_pred_prob = model.predict(X_test).flatten()
    y_pred = (y_pred_prob > 0.5).astype(int)

    acc  = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, zero_division=0)
    rec  = recall_score(y_test, y_pred, zero_division=0)
    f1   = f1_score(y_test, y_pred, zero_division=0)

    try:
        rocauc = roc_auc_score(y_test, y_pred_prob)
    except:
        rocauc = 0.0   # when only 1 class in test set

    print(f"ROC-AUC: {rocauc:.4f}")

    print(f"""
    🔹 Cluster {cl} Evaluation:
       Accuracy:  {acc:.4f}
       Precision: {prec:.4f}
       Recall:    {rec:.4f}
       Macro-F1:  {f1:.4f}
    """)



🔵 Training LSTM for Cluster 3
Cluster 3: Final sequence shape = (40538, 20, 23)
Epoch 1/30


c:\Users\kirut\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


178/178 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.5043 - loss: 0.7068 - val_accuracy: 0.5102 - val_loss: 0.6948
Epoch 2/30
178/178 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.5167 - loss: 0.6949 - val_accuracy: 0.5012 - val_loss: 0.6952
Epoch 3/30
178/178 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5198 - loss: 0.6938 - val_accuracy: 0.5000 - val_loss: 0.6942
Epoch 4/30
178/178 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5176 - loss: 0.6931 - val_accuracy: 0.4996 - val_loss: 0.6948
Epoch 5/30
178/178 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5225 - loss: 0.6922 - val_accuracy: 0.5004 - val_loss: 0.6938
Epoch 6/30
178/178 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5194 - loss: 0.6919 - val_accuracy: 0.4996 - val_loss: 0.6937
Epoch 7/30
178/178 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.5253 - loss: 0.6916 - val_accuracy: 0.5004 - val_loss: 0.6940
Epoch 8/30
178/178 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5242 - loss: 0.6918 - val_accuracy: 0.5004 - val_

c:\Users\kirut\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.5340 - loss: 0.7062 - val_accuracy: 0.5140 - val_loss: 0.7080
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5378 - loss: 0.6988 - val_accuracy: 0.5140 - val_loss: 0.7036
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5426 - loss: 0.6977 - val_accuracy: 0.5140 - val_loss: 0.7032
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5308 - loss: 0.6973 - val_accuracy: 0.5140 - val_loss: 0.7007
Epoch 5/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5375 - loss: 0.6949 - val_accuracy: 0.5140 - val_loss: 0.6991
Epoch 6/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5321 - loss: 0.6941 - val_accuracy: 0.5140 - val_loss: 0.6981
Epoch 7/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5493 - loss: 0.6916 - val_accuracy: 0.5140 - val_loss: 0.7022
Epoch 8/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5442 - loss: 0.6896 - val_accuracy: 0.5140 - val_loss: 0.6990
Ep

...

19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.5113 - loss: 0.7358 - val_accuracy: 0.4456 - val_loss: 0.6928
Epoch 2/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5045 - loss: 0.7089 - val_accuracy: 0.4456 - val_loss: 0.6930
Epoch 3/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4943 - loss: 0.7137 - val_accuracy: 0.5646 - val_loss: 0.6900
Epoch 4/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5121 - loss: 0.7008 - val_accuracy: 0.5646 - val_loss: 0.6900
Epoch 5/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5096 - loss: 0.7033 - val_accuracy: 0.5646 - val_loss: 0.6896
Epoch 6/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5262 - loss: 0.6971 - val_accuracy: 0.5646 - val_loss: 0.6910
Epoch 7/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4998 - loss: 0.7014 - val_accuracy: 0.5646 - val_loss: 0.6909
Epoch 8/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5006 - loss: 0.7017 - val_accuracy: 0.5646 - val_loss: 0.6901
Ep

c:\Users\kirut\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


68/68 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.5045 - loss: 0.7002 - val_accuracy: 0.5118 - val_loss: 0.6928
Epoch 2/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5012 - loss: 0.6969 - val_accuracy: 0.5123 - val_loss: 0.6931
Epoch 3/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5128 - loss: 0.6949 - val_accuracy: 0.5123 - val_loss: 0.6928
Epoch 4/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.5074 - loss: 0.6942 - val_accuracy: 0.4896 - val_loss: 0.6931
Epoch 5/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5096 - loss: 0.6945 - val_accuracy: 0.4891 - val_loss: 0.6933
Epoch 6/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5054 - loss: 0.6946 - val_accuracy: 0.5123 - val_loss: 0.6929
Epoch 7/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.5100 - loss: 0.6938 - val_accuracy: 0.5123 - val_loss: 0.6928
Epoch 8/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5079 - loss: 0.6940 - val_accuracy: 0.5109 - val_loss: 0.6930
14

c:\Users\kirut\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.5184 - loss: 0.6991 - val_accuracy: 0.5510 - val_loss: 0.6888
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.4871 - loss: 0.7034 - val_accuracy: 0.5510 - val_loss: 0.6888
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5171 - loss: 0.6958 - val_accuracy: 0.5510 - val_loss: 0.6895
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5353 - loss: 0.6941 - val_accuracy: 0.5510 - val_loss: 0.6882
Epoch 5/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5152 - loss: 0.6950 - val_accuracy: 0.5510 - val_loss: 0.6899
Epoch 6/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5110 - loss: 0.6946 - val_accuracy: 0.5510 - val_loss: 0.6903
Epoch 7/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5193 - loss: 0.6918 - val_accuracy: 0.5510 - val_loss: 0.6894
Epoch 8/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5174 - loss: 0.6943 - val_accuracy: 0.5510 - val_loss: 0.6897
E